<a href="https://colab.research.google.com/github/solozano0725/diplomadoMLNivel1/blob/main/Ejercicios_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 1**: Predicción de precios de diamantes

Este conjunto de datos contiene información de diferentes diamantes. Las características del conjunto de datos son:

* **price:** Precio (US) del diamante
* **carat** Peso del diamante
* **cut**: Calidad del corte (Fair, Good, Very Good, Premium, Ideal)
* **color**: Color del diamante, desde J (peor) hasta D (mejor)
* **clarity**: Una medida de claridad del diamante (I1 (peor), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (mejor))
* **x**: Longitud en mm 
* **y**: Ancho en mm 
* **z**: Profundidad en mm 
* **depth**: Porcentaje de profundidad total $\frac{2z}{x + y}$
* **table**: Ancho de la parte superior del diamante en relación con el punto más ancho

<p><img alt="Colaboratory logo" height="540px" src="https://i.imgur.com/FPgBl0y.jpg" align="left" hspace="10px" vspace="0px"></p> 

* Importe el conjunto de datos `diamonds.csv` del [repositorio](https://github.com/diplomadomludea/nivel_1) del curso y obtenga una primera descripción del mismo. ¿Hay valores faltantes en el conjunto de datos? 

para ver la solución haga doble click aquí

<!-- 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split, cross_val_score, learning_curve 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


diamonds = pd.read_csv("https://raw.githubusercontent.com/diplomadomludea/nivel_1/master/data/diamonds.csv", index_col=[0])

diamonds.head()

diamonds.info()

Observamos de esta información que el conjunto de datos no contiene valores faltantes
-->

* Defina una función `plot_cat_feature` que tome como argumento una característica categórica del conjunto de datos y que realice un gráfico de conteo y un gráfico de caja para ver la relación de la variable categórica con el precio. ¿Identifica valores aislados en el gráfico de caja? 

para ver la solución haga doble click aquí

<!-- 
def plot_cat_feature(feature):
  fig, ax = plt.subplots(1,2, figsize=(15,7))

  sns.countplot(data=diamonds, x=feature, ax=ax[0])
  sns.boxplot(data=diamonds, x=feature, y="price", ax=ax[1]);

plot_cat_feature("cut")
plot_cat_feature("clarity")
plot_cat_feature("color")

Las tres características "clarity", "cut" y "color" tienen un gran número de valores aislados, por lo que la mejor estrategia tal vez sea mantener estos datos.
-->

* Defina una función `plot_num_feature` que tome como argumento una característica numérica del conjunto de datos y muestre la distribución de la variable y un gráfico de dispersión para ver su relación con el precio. ¿Identifica valores aislados?

para ver la solución haga doble click aquí

<!-- 
def plot_num_feature(feature):
  fig, ax = plt.subplots(1,2, figsize=(15,7))

  sns.histplot(data=diamonds, x=feature, kde="True", ax=ax[0])
  sns.scatterplot(data=diamonds, x=feature, y="price", ax=ax[1]);

plot_num_feature("carat")
plot_num_feature("depth")
plot_num_feature("table")
plot_num_feature("x")
plot_num_feature("y")
plot_num_feature("z")

Las variables `depth`, `table`, `x`, `y` y `z` contienen valores aislados que pueden ser eliminados:

diamonds = diamonds.loc[diamonds.depth.between(45, 75)]
diamonds = diamonds.loc[diamonds.table.between(45, 80)]
diamonds = diamonds.loc[diamonds.x != 0]
diamonds = diamonds.loc[diamonds.y.between(0.1,20)]
diamonds = diamonds.loc[diamonds.z.between(2,10)]

visualicemos de nuevo las distribuciones:

plot_num_feature("depth")
plot_num_feature("table")
plot_num_feature("x")
plot_num_feature("y")
plot_num_feature("z")

¡Mucho mejor!
-->

* Visualice la correlación de las características con la variable objetivo ¿Qué características están más correlacionadas? ¿Qué características están menos correlacionadas? ¿Qué podría hacer en ambos casos?

para ver la solución haga doble click aquí

<!-- 
Utilizamos la correlación de `spearman` ya que las características no tienen distribuciones normales

# correlation
corr = diamonds.corr(method="spearman")

plt.figure(figsize=(11,8))
sns.heatmap(corr, vmin=-1, vmax=1, cmap="coolwarm", annot=True);

Vemos que Las características `depth` y `table` están muy poco correlacionadas con la variable objetivo. Estas no afectarán significativamente el modelo final (puede experimentar eliminando los datos y manteniéndolos). 

Las características `x`, `y` y `z` están altamente correlacionadas entre sí, por lo que podríamos construir una única variable que contenga la información de las tres características

# volume feature
diamonds["vol"] = diamonds.x * diamonds.y * diamonds.z

# dropping "x","y" and "z" features
diamonds.drop(columns=["x","y","z"], inplace=True)

# volume distribution
plot_num_feature("vol")

# correlation
plt.figure(figsize=(11,8))
sns.heatmap(corr, vmin=-1, vmax=1, cmap="coolwarm", annot=True);
-->

* Defina un preprocesador que realice una estandarización sobre las características numéricas y una codificación de las características categóricas. Evalúe el modelo mediante una validación cruzada y obtenga una curva de aprendizaje para examinar el comportamiento del modelo respecto al número de instancias. ¿Con cuál esquema de codificación se encuentra el mejor resultado? ¿El modelo subajusta o sobreajusta los datos en ambos casos?






para ver la solución haga doble click aquí

<!-- 
# feature matrix and label
X = diamonds.drop(columns="price")
y = diamonds["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# numerical and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

# ORDINAL ENCODING -----------------------------------------
preprocessor = make_column_transformer((StandardScaler(), num_features), 
                                       (OrdinalEncoder(), cat_features))

# model building
model = make_pipeline(preprocessor, LinearRegression())

# cross validation
cross_val_score(model, X_train, y_train)

Vemos que el modelo es estable y alcanza un puntaje R2 promedio de 0.88.

# learning curve
N, train_score, val_score = learning_curve(model, X_train, y_train, cv=5, train_sizes=np.linspace(0.1,1,30))

fig = plt.figure(figsize=(10,7))
plt.plot(N, np.mean(train_score, axis=1), "b", label='train score')
plt.plot(N, np.mean(val_score, axis=1), "r", label='test score')
plt.xlabel('training size')
plt.ylabel('score')
plt.legend();


# ONE-HOT ENCODING -----------------------------------------
preprocessor = make_column_transformer((StandardScaler(), num_features), 
                                       (OneHotEncoder(), cat_features))

model = make_pipeline(preprocessor, LinearRegression())

cross_val_score(model, X_train, y_train)

De nuevo, vemos que el modelo es estable y alcanza un puntaje R2 promedio de 0.91

N, train_score, val_score = learning_curve(model, X_train, y_train, cv=5, train_sizes=np.linspace(0.1,1,30))

# learning curve
fig = plt.figure(figsize=(10,7))
plt.plot(N, np.mean(train_score, axis=1), "b", label='train score')
plt.plot(N, np.mean(val_score, axis=1), "r", label='test score')
plt.xlabel('training size')
plt.ylabel('score')
plt.legend();

La codificación one-hot da mejores resultados y en ambos casos los modelos no presentan subajuste ni sobreajuste de acuerdo a las curvas de aprendizaje
-->

* Añada una transformación polinomial al preprocesador (para las características numéricas) y construya modelos de regresión polinomial de grados `[2,3,4,5]`. Para esto, defina una función `poly_reg` que tome como argumento el grado del polinomio y retorne el modelo de regresión polinomial. ¿Mejoran los resultados respecto a la regresión lineal? 

para ver la solución haga doble click aquí

<!-- 
def poly_reg(degree):
  """lineal regression model"""
  num_trans = make_pipeline(StandardScaler(), PolynomialFeatures(degree))

  preprocessor = make_column_transformer((num_trans, num_features), 
                                         (OneHotEncoder(), cat_features))
  
  return make_pipeline(preprocessor, LinearRegression())

for degree in [2,3,4,5]:
  model = poly_reg(degree).fit(X_train, y_train)
  print(f"degree: {degree}, test score: {model.score(X_test, y_test)}")

Para un grado tres se obtiene un puntaje de validación de 0.93, lo cual supone una leve mejora respecto a la regresión polinomial

# learning curve
N, train_score, val_score = learning_curve(poly_reg(3), X_train, y_train, cv=5, train_sizes=np.linspace(0.1,1,30))

fig = plt.figure(figsize=(10,7))
plt.plot(N, np.mean(train_score, axis=1), "b", label='train score')
plt.plot(N, np.mean(val_score, axis=1), "r", label='test score')
plt.xlabel('training size')
plt.ylabel('score')
plt.legend();

De la curva de validación vemos que el modelo está en un punto óptimo de sesgo/varianza

-->

# **Ejercicio 2:** Detección de spam

El conjunto de datos que vamos a utilizar en este ejercicio contiene alrededor de 5572 mensajes de texto (en inglés) etiquetados como *spam* o *no spam* (*ham*)

* Importe el conjunto de datos `spam.csv` del repositorio del curso. Los datos deben tener la siguiente forma

  ![](https://i.imgur.com/t43Yn2o.png)

  Consulte el parámetro `encoding` de la función `pd.read_csv`

para ver la solución haga doble click aquí

<!-- 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

data = pd.read_csv("https://raw.githubusercontent.com/diplomadomludea/nivel_1/master/data/spam.csv", encoding='latin-1', skipinitialspace=True, skiprows=[0], usecols=[0,1], names=["target","message"])

data.head()
-->

* ¿El conjunto de datos es desbalanceado? Realice un gráfico de barras para visualizar el número de instancias para cada clase.

para ver la solución haga doble click aquí

<!-- 
data.target.value_counts()

El conjunto de datos es desbalanceado: contiene 4825 instancias de la clase "ham" y 747 de la clase "spam"

plt.figure(figsize=(10,7))
sns.countplot(data=data, x="target");
-->

* Construya una nueva característica `length` que contenga el número de palabras por mensaje. 

para ver la solución haga doble click aquí

<!-- 
data['length'] = data.message.str.split().apply(len)
-->

* Realice un gráfico de densidad para visualizar la característica `length` distinguiendo las clases por un código de color. ¿Qué acción realizar a partir de la información que se obtiene de esta visualización?

para ver la solución haga doble click aquí

<!-- 
plt.figure(figsize=(10,7))
sns.kdeplot(data=data, x="length", hue="target", shade=True);

de aquí podemos ver que las instancias con un número de palabras mayor a 50 no son significativas. Eliminar estas instancias podría mejorar el rendimiento del modelo.

data = data.loc[data.length < 50]

plt.figure(figsize=(10,7))
sns.kdeplot(data=data, x="length", hue="target", shade=True);
-->

* Separe los datos en los conjuntos de entrenamiento y prueba y entrene tres modelos de clasificación utilizando Naive Bayes multinomial, SVM y regresión logística.

para ver la solución haga doble click aquí

<!-- 
X = data.message
y = data.target.replace({"spam":1, "ham":0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_clf = make_pipeline(TfidfVectorizer(), 
                       MultinomialNB()).fit(X_train, y_train)

svm_clf = make_pipeline(TfidfVectorizer(),
                        SVC()).fit(X_train, y_train)      

lr_clf = make_pipeline(TfidfVectorizer(),
                       LogisticRegression()).fit(X_train,y_train)                                           
-->

* Realice una validación cruzada para los tres clasificadores

para ver la solución haga doble click aquí

<!-- 
models = ["Naive Bayes", "SVM", "Logistic Regression"]
clfs = [nb_clf, svm_clf, lr_clf]

for model, clf in zip(models, clfs):
  print(f"{model}: {cross_val_score(clf, X, y, cv=5).mean()}")
-->

* Obtenga la matriz de confusión para cada clasificador

para ver la solución haga doble click aquí

<!-- 
for model, clf in zip(models, clfs):
  fig, ax = plt.subplots(figsize=(8, 5))
  plot_confusion_matrix(clf, X_test, y_test, values_format="d", cmap="Blues", ax=ax)
  ax.grid(None)
  ax.set_title(model);
-->

* Obtenga los puntajes de *precision*, *recall*, y *f score* para los tres clasificadores. Muestre estos resultados en un DataFrame.

para ver la solución haga doble click aquí

<!-- 
P = []
R = []
F = []

for clf in clfs:
  y_pred = clf.predict(X_test)
  P.append(precision_score(y_test, y_pred))
  R.append(recall_score(y_test, y_pred))
  F.append(f1_score(y_test, y_pred))

results = {"Precision":P,
           "Recall": R,
           "f score": F}

pd.DataFrame(results, index=models)
-->

* Obtenga las curvas ROC para cada clasificador con los datos de prueba

para ver la solución haga doble click aquí

<!-- 
fig, ax = plt.subplots(figsize=(12,8))

for model, clf in zip(models, clfs):
  plot_roc_curve(clf, X_test, y_test, label=model, ax=ax);
-->